# Using Chat Message History

In [1]:
from typing import Any, Dict, Optional
from langchain.prompts.chat import SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.chat_models.openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.memory.chat_message_histories import SQLChatMessageHistory
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable import (
    Runnable,
    RunnableBinding,
    RunnableConfig,
    RunnablePassthrough,
)

In [2]:
# Stateless chat

prompt = (
    SystemMessagePromptTemplate.from_template("You are a nice assistant.")
    + MessagesPlaceholder(variable_name="history")
    + "{question}"
)

openai = ChatOpenAI()

chain = prompt | openai | StrOutputParser()

chain.invoke({"question": "Hello, how are you?", "history": []})

"Hello! I'm an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"

In [3]:
# Chat with message history


class RunnableWithMessageHistory(RunnableBinding):
    input_key: str

    def __init__(self, runnable: Runnable, input_key: str) -> None:
        super().__init__(
            input_key=input_key,
            bound=(
                RunnablePassthrough.assign(history=self._enter_history)
                | runnable
                | RunnablePassthrough(self._exit_history)
            ),
            kwargs={},
        )

    def _enter_history(self, input: Dict[str, Any], config: RunnableConfig) -> None:
        hist: Optional[BaseChatMessageHistory] = config.get("configurable", {}).get(
            "message_history", None
        )
        if not hist:
            raise ValueError("No message history found in config.")

        messages = hist.messages

        hist.add_user_message(input[self.input_key])

        return messages

    def _exit_history(self, input: str, config: RunnableConfig) -> None:
        hist: Optional[BaseChatMessageHistory] = config.get("configurable", {}).get(
            "message_history", None
        )
        if not hist:
            raise ValueError("No message history found in config.")

        hist.add_ai_message(input)

In [4]:
chain_with_history = RunnableWithMessageHistory(chain, "question")


def invoke(question: str, session_id: str):
    return chain_with_history.invoke(
        {"question": question},
        {
            "configurable": {
                "message_history": SQLChatMessageHistory(
                    session_id=session_id, connection_string="sqlite:///test.db"
                )
            }
        },
    )

In [5]:
print(invoke("Hello, I'm Nuno?", "test"))

Hello Nuno! How can I assist you today?


In [6]:
print(invoke("What is my name?", "test"))

Your name is Nuno, as you mentioned earlier. Is there anything else you would like to know or discuss?
